# Implementation of U-Net architecture on Ultrasound Nerve Segmentation Kaggle Challenge

## This implementation is based on https://github.com/zhixuhao/unet

In [9]:
import os
import keras
from keras.models import Model
from keras.layers import Input,concatenate,Conv2D,MaxPooling2D,Conv2DTranspose
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K
import data

In [7]:
from skimage.transform import resize
from skimage.io import imsave
import numpy as np


In [3]:
#Loss Calculation. Here we use Dice loss as suggested in Competition
K.set_image_data_format('channels_last')  # TF dimension ordering in this code
smooth = 1.
def dice_coff(y_true,y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

![Architecture](Unet.jpg)

## Defining the U-Net Model

In [4]:
image_height = 96
image_width = 96

def UNet():
    
    #Declaring and Designing the Model https://keras.io/getting-started/functional-api-guide/
    
    inputs = Input((image_height,image_width,1))
    
    conv_1 = Conv2D(32,(3,3),activation='relu',padding='same')(inputs)
    conv_1 = conv2D(32,(3,3),activation='relu',padding='same')(conv_1)
    pool_1 = MaxPooling2D(pool_size=(2,2))(conv_1)
    
    conv_2 = Conv2D(64,(3,3),activation='relu',padding='same')(pool_1)
    conv_2 = conv2D(64,(3,3),activation='relu',padding='same')(conv_2)
    pool_2 = MaxPooling2D(pool_size=(2,2))(conv_2)
    
    conv_3 = Conv2D(128,(3,3),activation='relu',padding='same')(pool_2)
    conv_3 = conv2D(128,(3,3),activation='relu',padding='same')(conv_3)
    pool_3 = MaxPooling2D(pool_size=(2,2))(conv_3)
    
    conv_4 = Conv2D(256,(3,3),activation='relu',padding='same')(pool_3)
    conv_4 = conv2D(256,(3,3),activation='relu',padding='same')(conv_4)
    pool_4 = MaxPooling2D(pool_size=(2,2))(conv_4)
    
    conv_5 = Conv2D(512,(3,3),activation='relu',padding='same')(pool_4)
    conv_5 = conv2D(512,(3,3),activation='relu',padding='same')(conv_5)
    
    #Here I was little bit confused between Conv2DTranspose or UpSamplin2D with Conv2D
    #Explanation is here. 
    #https://stackoverflow.com/questions/48226783/what-is-the-the-difference-between-performing-upsampling-together-with-strided-t?rq=1
    #I am using Conv2DTranspose with kernel size = 2 and stride = 2
    
    #Concatenation is done on 3rd axis which is channel axis. 
    up_6 = concatenate([Conv2DTranspose(256,(2,2),strides=(2,2),padding='same')(conv_5),conv_4],axis=3)
    conv_6 = Conv2D(256,(3,3),activation='relu',padding='same')(up_6)
    conv_6 = Conv2D(256,(3,3),activation='relu',padding='same')(conv_6)
    
    up_7 = concatenate([Conv2DTranspose(128,(2,2),strides=(2,2),padding='same')(conv_6),conv_3],axis=3)
    conv_7 = Conv2D(128,(3,3),activation='relu',padding='same')(up_7)
    conv_7 = Conv2D(128,(3,3),activation='relu',padding='same')(conv_7)
    
    up_8 = concatenate([Conv2DTranspose(64,(2,2),strides=(2,2),padding='same')(conv_7),conv_2],axis=3)
    conv_8 = Conv2D(64,(3,3),activation='relu',padding='same')(up_8)
    conv_8 = Conv2D(64,(3,3),activation='relu',padding='same')(conv_8)
 
    up_9 = concatenate([Conv2DTranspose(32,(2,2),strides=(2,2),padding='same')(conv_8),conv_1],axis=3)
    conv_9 = Conv2D(32,(3,3),activation='relu',padding='same')(up_9)
    conv_9 = Conv2D(32,(3,3),activation='relu',padding='same')(conv_9)
    
    conv_10 = Conv2D(1,(1,1),activation='sigmoid')(conv_9)
    
    model = Model(inputs=[inputs],outputs=[conv_10])
    
    model.compile(optimizer = Adam(lr=1e-5),loss=dice_coef_loss,metrics=[dice_coef])

    return model






    

In [5]:
#Resize images to smaller size
def preprocess(images):
    images_resized = np.ndarray((images.shape[0],image_height,image_width),dtype=np.uint8)
    for i in range(images.shape[0]):
        images_resized[i] = resize(images[i],(image_height,image_width),preserve_range=True)
        
    images_resized = images_resized[...,np.newaxis]
    return images_resized

In [8]:
def train_and_predict():
    train_images,train_mask_images = data.load_train_data()
    
    train_images = preprocess(train_images)
    train_mask_images = preprocess(train_mask_images)
    
    train_images = train_images.astype('float32')
    mean = np.mean(train_images)
    std = np.std(train_images)
    
    train_images = train_images - mean
    train_images = train_images/255.0
    
    #Importing model
    model = UNet()
    model_checkpoint = ModelCheckpoint('weights.h5',monitor='val_loss',save_best_only=True)
    
    #Fitting the model
    model.fit(train_images,train_mask_images,batch_size = 32,nb_epoch=20,verbose=1,shuffle=True,validation_split=0.2,callbacks=[model_checkpoint])
    
    
    test_images,test_id_images = data.load_test_data()
    test_images = preprocess(test_images)
    
    test_images = test_images.astype('float32')
    test_images = test_images - mean
    test_images = test_images/std
    
    #Loading Saved weights
    model.load_weights('weights.h5')
    
    #Predicting masks on Test dataset
    test_mask_images = model.predict(test_images,verbose=1)
    np.save('test_mask.images.npy',test_mask_images)
    
    #Saving Predicted masks to file
    pred_dir = 'predictions'
    if not os.path.exists(pred_dir):
        os.mkdir(pred_dic)
    for image,image_id in zip(test_mask_images,test_id_images):
        image = (image[:,:,0]*255.).astype(np.uint8)
        imsave(os.path.join(pred_dir,str(image_id) + '_pred.png'),image)

In [ ]:
#DO NOT RUN THIS ON CPU.
##train_and_predict()